# TEST OF CONNECTIONS

## Basic initialization of the lorentzian manifold

Initialization of the Schwarzschild manifold $\mathcal M_{Schwarzschild}$, atlas with Schwarzschild chart (SD) and of the metric tensor $g$ :

In [1]:
%display latex

M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
#BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):\phi") #boyer-lindquist chart definition
SD.<t, r, th, ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")

var('m', domain='real')
#var('m, a', domain='real') #definition of Kerr metric parameters

g = M.metric() #we put the label g on the metric tensor

#rho2 = r^2 + (a*cos(th))^2
#Delta = r^2 -2*m*r + a^2

#g[0,0] = -(1-2*m*r/rho2)
#g[0,3] = -2*a*m*r*sin(th)^2/rho2
#g[1,1], g[2,2] = rho2/Delta, rho2
#g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g[0, 0] = - (1 - 2*m/r)
g[1, 1] = 1/(1 - 2*m/r)
g[2, 2] = r^2
g[3, 3] = r^2*sin(th)^2

g.display()

g = (2*m/r - 1) dt*dt - 1/(2*m/r - 1) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

## Initialization of the orthonormal tetrad

Definition of the orthonormal tetrad frame $e_{(a)}$ (Chandrasekar notation) and its dual frame $e^{(a)} = \eta^{(a)(b)}g(e_{(b)},-)$ :

In [2]:
e0 = M.vector_field(1/sqrt(1-2*m/r), 0, 0, 0, frame=SD.frame(), name='e_{(0)}')
e1 = M.vector_field(0, sqrt(1-2*m/r), 0, 0, frame=SD.frame(), name='e_{(1)}')
e2 = M.vector_field(0, 0, 1/r, 0, frame=SD.frame(), name='e_{(2)}')
e3 = M.vector_field(0, 0, 0, 1/(r*sin(th)), frame=SD.frame(), name='e_{(3)}')


e = M.vector_frame('e', (e0, e1, e2, e3)) #costruction of the tetrad collection object e_{a}
de = e.coframe() #construction of the tetrad coframe object e^{a}  


## ST bundle

Definition of the ST bundle:

In [3]:
from ST_bundle.ST_bundle import ST_bundle

In [4]:
STb = ST_bundle(M,e)

## Frame bundle connection $D_{fb}$ (tconnection)

The Frame bundle connection is a smart object that acts on objects with tindices and sindices. It that takes a tangent vector field, or the entire tetrad frame and gives as a result the frame bundle covariant derivative (tcovariant derivative) with respect to the tangent vector or the tetrad frame. It doesn't act on sindices.

The connection coefficents are the Ricci rotation coefficents $\omega^{(a)}_{\ \ \ \ (b)(c)}$

In [5]:
nabla_t = STb.tconnection() #definition of the tconnection
nabla_t.connection_coef #Ricci rotation coefficents

['up', 'down', 'down'] Tetrad index family of scalar fields on M with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

Example: calculation of the tcovariant derivative $S^{(a)}_{\ \ \ \ | (b)} = S^{(a)}_{\ \ \ \ , (b)} + \omega^{(a)}_{\ \ \ \ (c)(b)}S^{(c)}$

In [6]:
S = STb.scalar(["up"])
nabla_t(e,S)

['up', 'down'] Tetrad index family of scalar fields on M with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

Equivalenty one can use the method tcov_der() of the class _scalar:

In [7]:
nabla_t(e,S) == S.tcov_der()

True

Example: calculation of the tcovariant derivative with respect to the tangent vector V: $V^{(b)}S^{(a)}_{\ \ \ \ | (b)} = V^{(b)}(S^{(a)}_{\ \ \ \ , (b)} + \omega^{(a)}_{\ \ \ \ (c)(b)}S^{(c)})$

In [8]:
V = M.vector_field([1,0,0,0])
nabla_t(V,S)

['up'] Tetrad index family of scalar fields on M with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

Same works also for spinor tensors with tindices, tconnection doesn't see sindices though.
Here $\gamma^{(a)}_{\ \ \  |(b)}$

In [9]:
Gamma = STb.Dirac_matrices("up")
print(nabla_t(e,Gamma) == Gamma.tcov_der())
nabla_t(V,Gamma)

True


['up'] Tetrad index family of Tensors (1, 1) on the spin bundle section module with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

Connection works also on sections of the proper frame bundle, using the methods in Sage class BundleConnection, from which it inherits

In [10]:
F = STb.fbundle.section([1,0,0,0])
nabla_t(V,F) #-> returns a section
nabla_t(e,F) #-> returns a tindices object

['up', 'down'] Tetrad index family of scalar fields on M with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

tconnection inherits all the methods of the class BundleConnection, for example connection_form, curvature_form ...

In [11]:
nabla_t.connection_form(0,1).display()

connection (0,1) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = m/r^2 dt

In [12]:
nabla_t.curvature_form(0,1).display()

curvature (0,1) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = 2*m/r^3 dt/\dr

## Spinor bundle connection $D_{sb}$ (sconnection)

The Spinor bundle connection is a smart object that acts on objects with sindices and tindices. It takes a tangent vector field, or the entire tetrad frame and gives as a result the Spinor bundle covariant derivative (scovariant derivative) with respect to the tangent vector or the tetrad frame. It doesn't act on tindices.

The connection coefficents are by default $\Gamma_{(a)}= \frac{1}{4}\omega_{(b)(c)(a)}\gamma^{(b)}\gamma^{(c)}$, but the connection is up to the spinor identity times a one form (["down"] scalar tindices object), so in general is:
$\Gamma_{(a)}^{(A)}= \Gamma_{(a)} + IA_{(a)}$

In [13]:
nabla_s = STb.sconnection() #is we want to add the A field (tangent bundle one form) STb.connection(A)

Example: calculation of the tcovariant derivative $(\gamma^{(a)}_{\ \ \ \ : (b)})^{i} = (\gamma^{(a)}_{\ \ \ \ , (b)})^{i} + (\Gamma_{(b)})^{i}_{\ \ \ j}(\gamma^{(a)})^{j}$

In [14]:
nabla_s(e,Gamma) == Gamma.scov_der() #as before it is equivalent

True

Example: calculation of the scovariant derivative with respect to the tangent vector V: $V^{(b)}(\gamma^{(a)}_{\ \ \ \ : (b)})^{i} = V^{(b)}[(\gamma^{(a)}_{\ \ \ \ , (b)})^{i} + (\Gamma_{(b)})^{i}_{\ \ \ j}(\gamma^{(a)})^{j}]$

In [15]:
nabla_s(V,Gamma)

['up'] Tetrad index family of Tensors (1, 1) on the spin bundle section module with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

Connection works also on sections of the proper spinor bundle (spinor fields), using the methods in Sage class BundleConnection, from which it inherits:

In [16]:
psi = STb.sbundle.section([1,0,0,0])
nabla_s(V,psi) #-> returns a section
nabla_s(e,psi) #-> returns a tindices object

['down'] Tetrad index family of Tensors (1, 0) on the spin bundle section module with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

tconnection inherits all the methods of the class BundleConnection, for example connection_form, curvature_form ...

In [17]:
nabla_s.connection_form(0,1).display()

connection (0,1) of bundle connection \nabla_s w.r.t. Local frame (S|_M, (e_0,e_1,e_2,e_3)) = 1/2*(-I*sqrt(r)*cos(th) - sqrt(-2*m + r)*sin(th))/sqrt(r) dph

In [18]:
nabla_s.curvature_form(0,1).display()

curvature (0,1) of bundle connection \nabla_s w.r.t. Local frame (S|_M, (e_0,e_1,e_2,e_3)) = -1/2*m*sin(th)/(sqrt(-2*m + r)*r^(3/2)) dr/\dph + I*m*sin(th)/r dth/\dph

## Total connection $D_{ST}$ (totconnection)

The Total connection is a smart object that acts on objects with sindices and tindices. It takes a tangent vector field, or the entire tetrad frame and gives as a result the total covariant derivative (totcovariant derivative) with respect to the tangent vector or the tetrad frame. It acts on both tindices and sindices.

The connection coefficents are the union of the Spinor connection coefficents and the Ricci rotation coefficents

In [19]:
nabla_tot = STb.totconnection()

Example: calculation of the tcovariant derivative $(\gamma^{(a)}_{\ \ \ \ ! (b)})^{i} = (\gamma^{(a)}_{\ \ \ \ , (b)})^{i} + (\Gamma_{(b)})^{i}_{\ \ \ j}(\gamma^{(a)})^{j} + \omega^{(a)}_{\ \ \ \ (c)(b)}(\gamma^{(c)})^{i}$

In [20]:
nabla_tot(e,Gamma) == Gamma.totcov_der() #as before it is equivalent

True

Example: calculation of the scovariant derivative with respect to the tangent vector V: $V^{(b)}(\gamma^{(a)}_{\ \ \ \ : (b)})^{i} = V^{(b)}[(\gamma^{(a)}_{\ \ \ \ , (b)})^{i} + (\Gamma_{(b)})^{i}_{\ \ \ j}(\gamma^{(a)})^{j}+\omega^{(a)}_{\ \ \ \ (c)(b)}(\gamma^{(c)})^{i}]$

In [21]:
nabla_tot(V,Gamma)

['up', 'down'] Tetrad index family of Tensors (1, 1) on the spin bundle section module with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

Total connection is smart, in the sense that if we pass as argument an object with only tindices or a section of the frame bundle is acts only with tconnection. If we pass an object with only s indices or a section of the spinor bundle it acts only with sconnection:

In [22]:
print(nabla_tot(e,S) == nabla_t(e,S))
print(nabla_tot(e,F) == nabla_t(e,F))
print(nabla_tot(e,Gamma[0]) == nabla_s(e,Gamma[0]))
print(nabla_tot(e,psi) == nabla_s(e,psi))

True
True
True
True
